<a href="https://colab.research.google.com/github/mojcakloos/Kloos_Mojca_RL_gender_bias/blob/src/RL_gender_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies, import modules, load data

In [ ]:
#install dependencies
!pip install tensorflow
!pip install transformers
!pip install keras-rl2
!pip install pytorch_pretrained_bert

In [ ]:
#import libraries
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig, BertModel
from pytorch_pretrained_bert.optimization import BertAdam
from transformers import  BertForSequenceClassification
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from transformers import BertTokenizer
from sklearn import preprocessing
from nltk import word_tokenize

import pandas as pd
import json
import pickle
import math
from collections import defaultdict, Counter
from scipy.stats import pearsonr
import keras
import tensorflow as tf
import numpy as np
import ast
import matplotlib.pyplot as plt

from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Activation, Embedding
from keras.optimizers import Adam, SGD
from keras.layers import LSTM
import gym
from gym import error, spaces
from gym import utils
from gym.utils import seeding
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from PIL import Image
import keras.backend as K
from keras.optimizers import Adam
from tensorflow.python.keras.backend import set_session
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor


# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

In [ ]:
def load_data():
"""
Read in dataset, tokenize and pad biographies, transform to input ids for embedding layer
save input ids to file
encode labels

"""
  config = [255710, 194] #number of biographies, maxlength of sequence
  train_data = pd.read_pickle("/content/drive/MyDrive/thesis/data/train.pickle")
  dev_data = pd.read_pickle("/content/drive/MyDrive/thesis/data/dev.pickle")
  test_data = pd.read_pickle("/content/drive/MyDrive/thesis/data/test.pickle")
  with open("/content/drive/MyDrive/thesis/data/gender_distribution.txt", "r") as infile:
      reader = infile.read()
      distribution_dict = json.loads(reader)

  #set model and tokenizer: Both BERT pretrained
  model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True)
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
  le = preprocessing.LabelEncoder()
  MAX_LEN = 194

  #training set
  text_labels = []
  bios = []
  train_gender_list = []
  for bio in train_data:
    bios.append(bio["hard_text"])
    text_labels.append(bio["p"])
    if bio["g"] == "m":
      train_gender_list.append(0)
    else:
      train_gender_list.append(1)

  le.fit(text_labels)
  train_labels = le.transform(text_labels)

  sentences = []
  for bio in bios:
    sentence = "[CLS] " + bio + " [SEP]"
    sentences.append(sentence)


  tokenized_texts = []
  for i, sentence in enumerate(sentences):
    tokenized_texts.append(tokenizer.tokenize(sentence))


  # Pad our input tokens -> training dat
  train_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                                maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  #   # # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
  train_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  #save input ids to file
  np.save('/content/drive/MyDrive/thesis/data/train_input_ids.npy', np.array(train_input_ids, dtype=object), allow_pickle=True)

   #development set
  dev_text_labels = []
  dev_bios = []
  dev_gender_list = []
  for bio in dev_data:
    dev_bios.append(bio["hard_text"])
    dev_text_labels.append(bio["p"])
    if bio["g"] == "m":
      dev_gender_list.append(0)
    else:
      dev_gender_list.append(1)
  # le = preprocessing.LabelEncoder()
  le.fit(dev_text_labels)
  dev_labels = le.transform(dev_text_labels)

  dev_sentences = []
  for bio in dev_bios:
    sentence = "[CLS] " + bio + " [SEP]"
    dev_sentences.append(sentence)

  dev_tokenized_texts = []
  for i, sentence in enumerate(dev_sentences):
    dev_tokenized_texts.append(tokenizer.tokenize(sentence))
  # #pad dev data
  dev_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in dev_tokenized_texts],
                            maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  dev_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in dev_tokenized_texts]
  dev_input_ids = pad_sequences(dev_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  #save to file
  np.save('/content/drive/MyDrive/thesis/data/dev_input_ids.npy', np.array(dev_input_ids, dtype=object), allow_pickle=True)


  #test set
  test_bios = []
  test_text_labels = []
  test_gender_list = []
  for bio in test_data:
    test_bios.append(bio["hard_text"])
    test_text_labels.append(bio["p"])
    if bio["g"] == "m":
      test_gender_list.append(0)
    else:
      test_gender_list.append(1)

  le = preprocessing.LabelEncoder()
  le.fit(test_text_labels)
  test_labels = le.transform(test_text_labels)

  test_sentences = []
  for bio in test_bios:
    sentence = "[CLS] " + bio + " [SEP]"
    test_sentences.append(sentence)

  test_tokenized_texts = []
  for i, sentence in enumerate(test_sentences):
    test_tokenized_texts.append(tokenizer.tokenize(sentence))

#pad test data
  test_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in test_tokenized_texts],
                            maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  test_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in test_tokenized_texts]
  test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
#save to file
  np.save('/content/drive/MyDrive/thesis/data/test_input_ids.npy', np.array(test_input_ids, dtype=object), allow_pickle=True)

In [ ]:
# if running for the first time, uncomment the next line to preprocess all data
# load_data()

In [ ]:
#if you have saved input ids, load all inputs from file
train = pd.read_pickle("/content/drive/MyDrive/thesis/data/train.pickle")
dev = pd.read_pickle("/content/drive/MyDrive/thesis/data/dev.pickle")
test = pd.read_pickle("/content/drive/MyDrive/thesis/data/test.pickle")
all_data = train+dev+test

x_train = np.load('/content/drive/MyDrive/thesis/data/train_input_ids.npy', allow_pickle =True)
x_dev =np.load('/content/drive/MyDrive/thesis/data/dev_input_ids.npy', allow_pickle =True)
x_test =np.load('/content/drive/MyDrive/thesis/data/test_input_ids.npy', allow_pickle =True)

with open ('/content/drive/MyDrive/thesis/data/train_labels', 'rb') as fp:
    y_train = pickle.load(fp)
with open ('/content/drive/MyDrive/thesis/data/dev_labels', 'rb') as fp:
    y_dev = pickle.load(fp)
with open ('/content/drive/MyDrive/thesis/data/test_labels', 'rb') as fp:
    y_test = pickle.load(fp)
with open ('/content/drive/MyDrive/thesis/data/gender_list', 'rb') as fp:
    gender_list = pickle.load(fp)
# with open ('/content/drive/MyDrive/thesis/data/dev_gender_list', 'rb') as fp:
#     dev_gender_list = pickle.load(fp)
with open ('/content/drive/MyDrive/thesis/data/test_gender_list', 'rb') as fp:
    test_gender_list = pickle.load(fp)
with open("/content/drive/MyDrive/thesis/data/gender_distribution.txt", "r") as infile:
    reader = infile.read()
    distribution_dict = json.loads(reader)

## code for evaluating models

In [ ]:
#code inspired by https://github.com/tue-alga/debias-mean-projection/tree/main/notebooks

def load_dictionary(path):

    with open(path, "r", encoding = "utf-8") as f:

        lines = f.readlines()

    k2v, v2k = {}, {}
    for line in lines:

        k,v = line.strip().split("\t")
        v = int(v)
        k2v[k] = v
        v2k[v] = k

    return k2v, v2k

def count_profs_and_gender(data):
  # count number of individuals with a specific gender in every occupation
  # returns dictionary
    counter = defaultdict(Counter)
    for entry in data:
        gender, prof = entry["g"], entry["p"]
        counter[prof][gender] += 1

    return counter

def get_prof2fem(all_data):

  counter = count_profs_and_gender(train+dev+test)
  f,m = 0., 0.
  prof2fem = dict()

  for k, values in counter.items():
      f += values['f']
      m += values['m']
      prof2fem[k] = values['f']/(values['f'] + values['m'])
  return(prof2fem)

In [ ]:
p2i, i2p = load_dictionary('/content/drive/MyDrive/thesis/data/profession2index.txt')
prof2fem = get_prof2fem(test)

In [ ]:
#code for evaluation metrics, modified from https://github.com/tue-alga/debias-mean-projection/tree/main/notebooks

def get_tpr(y_pred, y_true, p2i, i2p, test_gender_list):
    scores = defaultdict(Counter)
    prof_count_total = defaultdict(Counter)

    for y_hat, y, g in zip(y_pred, y_true, test_gender_list):
      if int(y) == y_hat:
        scores[(i2p[y])][g] +=1
      prof_count_total[(i2p[y])][g] +=1

    tpr_dict = defaultdict(dict)
    tpr_change = dict()
    tpr_ratio = []

    for profession, scores_dict in scores.items():

        good_m, good_f = scores_dict[0], scores_dict[1]
        prof_total_f = prof_count_total[profession][1]
        prof_total_m = prof_count_total[profession][0]
        tpr_m = (good_m) / prof_total_m
        tpr_f = (good_f) / prof_total_f

        tpr_dict[profession][0] = tpr_m
        tpr_dict[profession][1] = tpr_f
        tpr_ratio.append(0)
        tpr_change[profession] = tpr_f - tpr_m
    return tpr_dict, tpr_change, np.mean(np.abs(tpr_ratio))

def similarity_vs_tpr(tpr_dict, title, measure, prof2fem):

    professions = list(tpr_dict.keys())

    tpr_lst = [tpr_dict[p] for p in professions]
    sim_lst = [prof2fem[p] for p in professions]


    plt.plot(sim_lst, tpr_lst, marker = "o", linestyle = "none")
    plt.xlabel("% women", fontsize = 13)
    plt.ylabel(r'$GAP_{female,y}^{TPR}$', fontsize = 13)
    for p in professions:
        x,y = prof2fem[p], tpr_dict[p]
        plt.annotate(p , (x,y), size = 7, color = "red")

    plt.ylim(-1,1)
    z = np.polyfit(sim_lst, tpr_lst, 1)
    p = np.poly1d(z)
    plt.plot(sim_lst,p(sim_lst),"b--")
    print("Correlation: {}; p-value: {}".format(*pearsonr(sim_lst, tpr_lst)))

#MK: function that calculates aggregated TPR scores

def calc_tpr_gap(tpr_gaps):
    n = len(tpr_gaps) # number of occupations
    sum_of_squares = sum([gap**2 for gap in tpr_gaps.values()]) # sum of squares of TPR-GAPs
    rms = math.sqrt(sum_of_squares/n) # root mean square error
    return rms

# Vanilla classifier without RL

In [ ]:
def get_model(input_shape,output):
    top_words, max_words = input_shape
    model = Sequential()
    model.add(Embedding(top_words, 128, input_length=max_words))
    model.add(Flatten())
    model.add(Dense(250))
    model.add(Activation('relu'))
    model.add(Dense(output))
    model.add(Activation("softmax"))
    return model

In [ ]:
#define shape of data [number of biographies, max len of biography]
in_shape = [255710, 194]
num_classes = 28
#set seed
np.random.seed(50)
model = get_model(in_shape, num_classes)
optimizer = keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer, metrics=["accuracy"])

In [ ]:
#train and save the model
model.fit(x_train, y_train, epochs = 3, batch_size = 64)
model.save('/content/drive/MyDrive/thesis/bl1_5')

In [ ]:
#load pretrained model
pretrained_bl = tf.keras.models.load_model('/content/drive/MyDrive/thesis/bl1_1')

In [ ]:
#predict on test set
y_pred = model.predict(x_test)
y_pred=np.argmax(y_pred,axis=1)

#write predictions to file
with open('/content/drive/MyDrive/thesis/results/bl1_pred_1.txt', "w") as outfile:
  for item in y_pred:
    outfile.write(str(item)+"\n")

## evaluate

In [ ]:
#load predictions
pred_bl1 = []
with open('/content/drive/MyDrive/thesis/results/bl1_pred_1.txt', "r") as infile:
  predictions = infile.read()
  preds_tolist = predictions.split("\n")
  for pred in preds_tolist:
    if pred != "":
      pred_bl1.append(int(pred))

tpr_scores, tpr_gaps, tpr_mean_ratio = get_tpr(pred_bl1, y_test, p2i, i2p, test_gender_list)

print("Overall TPR Gap:"+str(calc_tpr_gap(pred_bl1, y_test)))

print("correlation")
similarity_vs_tpr(tpr_scores, "TPR scores BL 1", "TPR", prof2fem)

print("classification report")
print(classification_report(pred_bl1, y_test))


#RL with reward for correct classification


In [ ]:
#code taken from https://github.com/linenus/DRL-For-imbalanced-Classification
#train and save model
class ClassifyEnv(gym.Env):
    def __init__(self, mode, x_train, y_train, gender_list, distribution_dict):  # mode means training or testing
        self.mode = mode
        self.Env_data = x_train
        self.Answer = y_train
        self.id = np.arange(x_train.shape[0])
        self.game_len = self.Env_data.shape[0]
        self.num_classes = len(set(self.Answer))
        self.action_space = spaces.Discrete(self.num_classes)
        self.step_ind = 0
        self.y_pred = []

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(14)
        # print(f"SEED:{seed}")
        return [seed]

    def step(self, a):
        self.y_pred.append(a)
        y_true_cur = []
        info = {}
        terminal = False
        # MK:give positive reward for correct classification, negative for wrong prediction
        if a == int(self.Answer[self.id[self.step_ind]]):
          reward = 1
        else:
          reward = -1

        self.step_ind += 1

        # #update the info dict
        if self.step_ind == self.game_len - 1:
            terminal = True
        return self.Env_data[self.id[self.step_ind]], reward, terminal, info

    def My_metrics(self, y_pre, y_true):
        print(classification_report(y_true, y_pre))
        return(accuracy_score(y_true, y_pre))


    def reset(self):
        if self.mode == 'train':
            np.random.shuffle(self.id)
        self.step_ind = 0
        self.y_pred = []
        return self.Env_data[self.id[self.step_ind]]

num_training_steps = 1022840
num_classes = len(set(y_test))
mode = 'train'
env = ClassifyEnv(mode,x_train, y_train, gender_list, distribution_dict)
nb_actions = num_classes
training_steps = num_training_steps
in_shape = [255710, 100]
model = get_rl_model(in_shape, num_classes)



INPUT_SHAPE = in_shape


class ClassifyProcessor(Processor):
    def process_observation(self, observation):
        return observation

    def process_state_batch(self, batch):
        return batch.reshape((-1, INPUT_SHAPE[1]))

    def process_reward(self, reward):
        return reward


memory = SequentialMemory(limit=100000, window_length=1)
processor = ClassifyProcessor()
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=100000)
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
              processor=processor, nb_steps_warmup=50000, gamma=0.5, target_model_update=10000,
              train_interval=4, delta_clip=1.)
dqn.compile(Adam(learning_rate=.0001), metrics=['accuracy'])

dqn.fit(env, nb_steps=training_steps, log_interval=10000, nb_max_episode_steps =255709)

#save model to file
model.save('/content/drive/MyDrive/thesis/baseline2/bl2_1')

In [ ]:
#load pretrained model
bl2_1 = tf.keras.models.load_model('/content/drive/MyDrive/thesis/baseline2/bl2_1')


dqn_bl2_1= DQNAgent(model=bl2_1, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=0.5, target_model_update=10000,
               train_interval=4, delta_clip=1.)
pred_bl2 = []
#predict on test set
for i,bio in enumerate(x_test):
  pred = dqn_bl2_1.forward(bio)
  pred_bl2.append(pred)

#write predictions to file
with open('/content/drive/MyDrive/thesis/results/bl2_pred_1.txt', "w") as outfile:
  for item in pred_bl2:
    outfile.write(str(item)+"\n")

## evaluate

In [ ]:
#load predictions from file
pred_bl2 = []
with open('/content/drive/MyDrive/thesis/results/bl2_pred_1.txt', "r") as infile:
  predictions = infile.read()
  preds_tolist = predictions.split("\n")
  for pred in preds_tolist:
    if pred != "":
      pred_bl2.append(int(pred))

tpr_scores, tpr_gaps, tpr_mean_ratio = get_tpr(pred_bl2, y_test, p2i, i2p, test_gender_list)

print("Overall TPR Gap:"+str(calc_tpr_gap(pred_bl2, y_test)))

print("correlation")
similarity_vs_tpr(tpr_scores, "TPR scores BL 2", "TPR", prof2fem)

print("classification report")
print(classification_report(pred_bl2, y_test))

## Additional experiment: RL with reward for correct classification + scrubbed input

In [ ]:
def load_data_nogender():

  config = [255710, 194] #number of biographies, maxlength of sequence
  train_data = pd.read_pickle("/content/drive/MyDrive/thesis/data/train.pickle")
  # dev_data = pd.read_pickle("/content/drive/MyDrive/thesis/data/dev.pickle")
  test_data = pd.read_pickle("/content/drive/MyDrive/thesis/data/test.pickle")
  # with open("/content/drive/MyDrive/thesis/data/gender_distribution.txt", "r") as infile:
  #     reader = infile.read()
  #     distribution_dict = json.loads(reader)

  #set tokenizer: Both BERT pretrained
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
  le = preprocessing.LabelEncoder()
  MAX_LEN = 194

  #training set
  text_labels = []
  bios = []
  train_gender_list = []
  for bio in train_data:
    bios.append(bio["text_without_gender"])
    text_labels.append(bio["p"])
    if bio["g"] == "m":
      train_gender_list.append(0)
    else:
      train_gender_list.append(1)

  le.fit(text_labels)
  train_labels = le.transform(text_labels)

  sentences = []
  for bio in bios:
    sentence = "[CLS] " + bio + " [SEP]"
    sentences.append(sentence)


  tokenized_texts = []
  for i, sentence in enumerate(sentences):
    tokenized_texts.append(tokenizer.tokenize(sentence))


  # Pad our input tokens -> training dat
  train_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                                maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  #   # # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
  train_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  tokens_tensor = torch.tensor([train_input_ids])

  np.save('/content/drive/MyDrive/thesis/data/train_inputs_nogender.npy', np.array(train_input_ids, dtype=object), allow_pickle=True)



#   #test set
  test_bios = []
  test_text_labels = []
  test_gender_list = []
  for bio in test_data:
    test_bios.append(bio["text_without_gender"])
    test_text_labels.append(bio["p"])
    if bio["g"] == "m":
      test_gender_list.append(0)
    else:
      test_gender_list.append(1)

  le = preprocessing.LabelEncoder()
  le.fit(test_text_labels)
  test_labels = le.transform(test_text_labels)

  test_sentences = []
  for bio in test_bios:
    sentence = "[CLS] " + bio + " [SEP]"
    test_sentences.append(sentence)

  test_tokenized_texts = []
  for i, sentence in enumerate(test_sentences):
    test_tokenized_texts.append(tokenizer.tokenize(sentence))

#pad test data
  test_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in test_tokenized_texts],
                            maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  test_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in test_tokenized_texts]
  test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  np.save('/content/drive/MyDrive/thesis/data/test_inputs_nogender.npy', np.array(test_input_ids, dtype=object), allow_pickle=True)

In [ ]:
load_data_nogender()

In [ ]:
x_train_nogender = np.load('/content/drive/MyDrive/thesis/data/train_inputs_nogender.npy', allow_pickle =True)
x_test_nogender =np.load('/content/drive/MyDrive/thesis/data/test_inputs_nogender.npy', allow_pickle =True)

In [ ]:
class ClassifyEnv(gym.Env):
    def __init__(self, mode, x_train, y_train, gender_list, distribution_dict):  # mode means training or testing
        self.mode = mode
        self.Env_data = x_train
        self.Answer = y_train
        self.id = np.arange(x_train.shape[0])
        self.game_len = self.Env_data.shape[0]
        self.num_classes = len(set(self.Answer))
        self.action_space = spaces.Discrete(self.num_classes)
        self.step_ind = 0
        self.y_pred = []

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(14)
        # print(f"SEED:{seed}")
        return [seed]

    def step(self, a):
        self.y_pred.append(a)
        y_true_cur = []
        info = {}
        terminal = False
        #give positive reward for correct classification, negative for wrong prediction
        if a == int(self.Answer[self.id[self.step_ind]]):
          reward = 1
        else:
          reward = -1
          # if self.mode == 'train':
          #           terminal = True



        self.step_ind += 1
        # #update the info dict
        if self.step_ind == self.game_len - 1:
        #     y_true_cur = self.Answer[self.id]
        #     info['gmean'], info['fmeasure'] = self.My_metrics(np.array(self.y_pred),
        #                                                       np.array(y_true_cur[:self.step_ind]))
            terminal = True
        return self.Env_data[self.id[self.step_ind]], reward, terminal, info

    def My_metrics(self, y_pre, y_true):
        print(classification_report(y_true, y_pre))
        return(accuracy_score(y_true, y_pre))

    # return: (states, observations)
    def reset(self):
        if self.mode == 'train':
            np.random.shuffle(self.id)
        self.step_ind = 0
        self.y_pred = []
        return self.Env_data[self.id[self.step_ind]]

num_training_steps = 1022840
num_classes = len(set(y_test))
mode = 'train'
env = ClassifyEnv(mode,x_train_nogender, y_train, gender_list, distribution_dict)
nb_actions = num_classes
training_steps = num_training_steps
# np.random.seed(14)
in_shape = [255710, 194]
model = get_rl_model(in_shape, num_classes)



INPUT_SHAPE = in_shape


class ClassifyProcessor(Processor):
    def process_observation(self, observation):
        return observation

    def process_state_batch(self, batch):
        return batch.reshape((-1, INPUT_SHAPE[1]))

    def process_reward(self, reward):
        return reward


memory = SequentialMemory(limit=100000, window_length=1)
processor = ClassifyProcessor()
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=100000)
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
              processor=processor, nb_steps_warmup=50000, gamma=0.5, target_model_update=10000,
              train_interval=4, delta_clip=1.)
dqn.compile(Adam(learning_rate=.0001), metrics=['accuracy'])

dqn.fit(env, nb_steps=training_steps, log_interval=10000, nb_max_episode_steps =255709)
model.save(f'/content/drive/MyDrive/thesis/baseline2/bl2_nogender')

In [ ]:
bl2_nogender = tf.keras.models.load_model('/content/drive/MyDrive/thesis/baseline2/bl2_nogender')
dqn_bl2_nogender= DQNAgent(model=bl2_nogender, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=0.5, target_model_update=10000,
               train_interval=4, delta_clip=1.)

pred_m2 = []
for i,bio in enumerate(x_test_nogender):
  pred = dqn_bl2_nogender.forward(bio)
  pred_m2.append(pred)

with open('/content/drive/MyDrive/thesis/results/bl2_pred_nogender.txt', "w") as outfile:
  for item in pred_m2:
    outfile.write(str(item)+"\n")

## evaluate additional experiment

In [ ]:
#load predictions
pred_bl2_nogender = []
with open('/content/drive/MyDrive/thesis/results/bl2_pred_nogender.txt', "r") as infile:
  predictions = infile.read()
  preds_tolist = predictions.split("\n")
  for pred in preds_tolist:
    if pred != "":
      pred_bl2_nogender.append(int(pred))

tpr_scores, tpr_gaps, tpr_mean_ratio = get_tpr(pred_bl2_nogender, y_test, p2i, i2p, test_gender_list)

print("Overall TPR Gap:"+str(calc_tpr_gap(pred_bl2_nogender, y_test)))

print("correlation")
similarity_vs_tpr(tpr_scores, "TPR scores BL 2", "TPR", prof2fem)

print("classification report")
print(classification_report(pred_bl2_nogender, y_test))

# RL with minority-sensitive classification

In [ ]:
class ClassifyEnv(gym.Env):
    def __init__(self, mode, x_train, y_train, gender_list, distribution_dict):  # mode means training or testing
        self.mode = mode
        self.Env_data = x_train
        self.Answer = y_train
        self.id = np.arange(x_train.shape[0])
        self.game_len = self.Env_data.shape[0]
        self.num_classes = len(set(self.Answer))
        self.action_space = spaces.Discrete(self.num_classes)
        self.step_ind = 0
        self.y_pred = []

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(50)
        return [seed]

    def step(self, a):
        self.y_pred.append(a)
        y_true_cur = []
        info = {}
        terminal = False


        if a == int(self.Answer[self.id[self.step_ind]]): #if prediction is correct
          for k,v in distribution_dict.items():
            if a ==int(k):
              if int(gender_list[self.id[self.step_ind]]) == v["minority"]: #if the gender of the current bio is the minority gender
                # reward = 1
                reward = 1-v["perc_minority"] #reward is equal to 1 minus the percentage of the minority group
              else:
                reward = 0.5 #if the gender is in the majority group, the reward is 0.5

        else: #if the prediction is incorrect
          for k,v in distribution_dict.items():
            if a == int(k):
              if int(gender_list[self.id[self.step_ind]]) == v["minority"]: #if the gender of the current bio is minority gender
                reward = -(1-v["perc_minority"]) #agent gets higher punishment
                # reward = -1
                if self.mode == "train":
                  terminal = True #stop the episode
              else:
                reward = -0.5 #agent gets punishment only for wrong prediciton

        self.step_ind += 1
        #update the info dict
        if self.step_ind == self.game_len - 1:
            # y_true_cur = self.Answer[self.id]
            # info['gmean'], info['fmeasure'] = self.My_metrics(np.array(self.y_pred),
            #                                                   np.array(y_true_cur[:self.step_ind]))
            terminal = True
        return self.Env_data[self.id[self.step_ind]], reward, terminal, info

    def My_metrics(self, y_pre, y_true):
        print(classification_report(y_true, y_pre))
        return(accuracy_score(y_true, y_pre))

    # return: (states, observations)
    def reset(self):
        if self.mode == 'train':
            np.random.shuffle(self.id)
        self.step_ind = 0
        self.y_pred = []
        return self.Env_data[self.id[self.step_ind]]

In [ ]:
num_training_steps = 1022840

num_classes = len(set(y_test))
mode = 'test'
env = ClassifyEnv(mode,x_train, y_train, gender_list, distribution_dict)
nb_actions = num_classes
training_steps = num_training_steps

in_shape = [255710, 194]


INPUT_SHAPE = in_shape


class ClassifyProcessor(Processor):
    def process_observation(self, observation):
        return observation

    def process_state_batch(self, batch):
        return batch.reshape((-1, INPUT_SHAPE[1]))

    def process_reward(self, reward):
        return reward


memory = SequentialMemory(limit=100000, window_length=1)
processor = ClassifyProcessor()
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=100000)
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=0.5, target_model_update=10000,
               train_interval=4, delta_clip=1.)
dqn.compile(Adam(learning_rate=.0001), metrics=['accuracy'])

dqn.fit(env, nb_steps=training_steps, log_interval=10000)

#save model to file
model.save('/content/drive/MyDrive/thesis/main_rl_1')

In [ ]:
pred_main_1 = []
for bio in x_test:
  pred = dqn.forward(bio)
  pred_main_1.append(pred)

#write predictions to file
with open('/content/drive/MyDrive/thesis/results/main_pred_1.txt', "w") as outfile:
  for item in pred_main_1:
    outfile.write(str(item)+"\n")

In [ ]:
#load predictions
pred_main = []
with open('/content/drive/MyDrive/thesis/results/main_pred_1.txt', "r") as infile:
  predictions = infile.read()
  preds_tolist = predictions.split("\n")
  for pred in preds_tolist:
    if pred != "":
      pred_main.append(int(pred))

tpr_scores, tpr_gaps, tpr_mean_ratio = get_tpr(pred_main, y_test, p2i, i2p, test_gender_list)

print("Overall TPR Gap:"+str(calc_tpr_gap(pred_main, y_test)))

print("correlation")
similarity_vs_tpr(tpr_scores, "TPR scores Main 1", "TPR", prof2fem)

print("classification report")
print(classification_report(pred_main, y_test))